In [6]:
import torch, torch.nn.functional as F
import torch_geometric
print(torch.__version__, torch_geometric.__version__)


2.8.0+cpu 2.7.0


In [3]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root=r"C:\Users\Administrator\Desktop\GNNmodel\data\Planetoid", name="Cora", transform=NormalizeFeatures())
data = dataset[0]

1、定义并训练 GCN

In [4]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn. Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 =GCNConv(dataset.num_features, hidden_channels)#1433,16
        self.conv2 =GCNConv(hidden_channels, dataset.num_classes)#16,7

    def forward(self,x, edge_index):
        x=self.conv1(x,edge_index)#firstGCN，x---2708x1433的x，edge_index是A
        x=x.relu()
        x=F.dropout(x, p=0.5, training=self.training)
        x= self.conv2(x, edge_index)#second GCN
        return x
    
model = GCN(hidden_channels=16)
print (model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


2、训练与测试函数 + 训练 200 epoch，Test Accuracy

In [7]:
model = GCN(hidden_channels=16)
criterion = torch.nn.CrossEntropyLoss() # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01,weight_decay=5e-4)# Define optimizer.

def train():
    model.train()
    optimizer.zero_grad()#梯度清零
    out= model(data.x,data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()#梯度更新
    return loss

def test():
    model.eval()
    out = model(data.x,data.edge_index)
    pred =out.argmax(dim=1)#Use the class with highest probability.
    test_correct = pred[data.test_mask]== data.y[data.test_mask] # Check against ground-truth labels.
    test_acc = int(test_correct.sum())/ int(data.test_mask.sum())# Derive ratio of correct predictions
    return test_acc

for epoch in range(1,201):
    loss = train()
    print(f'Epoch:{epoch:03d},Loss:{loss:.4f}')

test_acc=test()
print(f'Test Accuracy:{test_acc:.4f}')

Epoch:001,Loss:1.9465
Epoch:002,Loss:1.9419
Epoch:003,Loss:1.9363
Epoch:004,Loss:1.9290
Epoch:005,Loss:1.9199
Epoch:006,Loss:1.9140
Epoch:007,Loss:1.9079
Epoch:008,Loss:1.8992
Epoch:009,Loss:1.8876
Epoch:010,Loss:1.8764
Epoch:011,Loss:1.8656
Epoch:012,Loss:1.8626
Epoch:013,Loss:1.8460
Epoch:014,Loss:1.8329
Epoch:015,Loss:1.8225
Epoch:016,Loss:1.8167
Epoch:017,Loss:1.7995
Epoch:018,Loss:1.7878
Epoch:019,Loss:1.7716
Epoch:020,Loss:1.7568
Epoch:021,Loss:1.7563
Epoch:022,Loss:1.7342
Epoch:023,Loss:1.7092
Epoch:024,Loss:1.7015
Epoch:025,Loss:1.6671
Epoch:026,Loss:1.6757
Epoch:027,Loss:1.6609
Epoch:028,Loss:1.6355
Epoch:029,Loss:1.6339
Epoch:030,Loss:1.6102
Epoch:031,Loss:1.5964
Epoch:032,Loss:1.5721
Epoch:033,Loss:1.5570
Epoch:034,Loss:1.5445
Epoch:035,Loss:1.5093
Epoch:036,Loss:1.4889
Epoch:037,Loss:1.4776
Epoch:038,Loss:1.4704
Epoch:039,Loss:1.4263
Epoch:040,Loss:1.3972
Epoch:041,Loss:1.3873
Epoch:042,Loss:1.3479
Epoch:043,Loss:1.3485
Epoch:044,Loss:1.3739
Epoch:045,Loss:1.3343
Epoch:046,

3、安装导入“解释器”所需组件

In [8]:
# PyG Explainability imports
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.explain.config import ModelConfig

# 常用工具：子图抽取、可视化需要
from torch_geometric.utils import k_hop_subgraph, to_networkx

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


4、创建 Explainer（用于边 mask）

In [10]:
import torch
import torch.nn.functional as F

from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.explain.config import ModelConfig

# 1) 让模型进入 eval（解释时通常不希望 dropout 影响结果）
model.eval()

# 2) 告诉 Explainer：这是“多分类节点分类”任务，模型输出是 logits
model_config = ModelConfig(
    mode='multiclass_classification',
    task_level='node',
    return_type='raw',   # raw = logits（还没 softmax）
)

# 3) 建立 explainer：这里用 GNNExplainer，先只做 edge mask
explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type=None,     # 先不做特征解释
    edge_mask_type='object', # 做边解释：给每条边一个重要性权重
    model_config=model_config,
)
print(explainer)


5、解释一个节点（得到 edge_mask）

In [11]:
import torch

# 1) 选一个测试集节点
node_id = int(torch.where(data.test_mask)[0][0])
print("node_id =", node_id)

# 2) 生成解释（只针对这个 node_id）
explanation = explainer(
    x=data.x,
    edge_index=data.edge_index,
    index=node_id,   # 解释哪个节点的预测
)

# 3) 检查解释里是否有 edge_mask
print("has edge_mask:", explanation.edge_mask is not None)
print("edge_mask shape:", tuple(explanation.edge_mask.shape))
print("edge_index shape:", tuple(data.edge_index.shape))


node_id = 1708
has edge_mask: True
edge_mask shape: (10556,)
edge_index shape: (2, 10556)


6、抽取 node_id=1708 的 Top-k 关键边（只看与该节点相连的边）

In [12]:
import torch

# 你现在解释的节点
node_id = 1708
top_k = 15  # 你可以改 10/20 都行

# 1) 取出 edge_index 和 edge_mask
edge_index = data.edge_index
edge_mask = explanation.edge_mask  # 如果你变量不是 explanation，请改成 ex 或 ex_bad

# 2) 找到“与 node_id 相连”的边（入边+出边都算 incident edges）
src, dst = edge_index[0], edge_index[1]
incident = (src == node_id) | (dst == node_id)
incident_eids = incident.nonzero(as_tuple=False).view(-1)  # 这些是边在 edge_index 里的索引

print(f"node_id={node_id}, incident edges count = {incident_eids.numel()}")

# 3) 取出这些 incident edges 的解释权重，并排序
incident_scores = edge_mask[incident_eids]
sorted_idx = torch.argsort(incident_scores, descending=True)
top_eids = incident_eids[sorted_idx[:top_k]]
top_scores = edge_mask[top_eids]

# 4) 打印 Top-k 关键边（边的两端节点 + 权重 + 两端节点真实标签）
print(f"\n=== node {node_id} Top-{top_k} incident edges by edge_mask ===")
for rank, (eid, score) in enumerate(zip(top_eids.tolist(), top_scores.tolist()), start=1):
    s = int(src[eid])
    t = int(dst[eid])
    ys = int(data.y[s])
    yt = int(data.y[t])
    print(f"{rank:02d}. eid={eid:5d}  ({s:4d} -> {t:4d})  w={score:.4f}   y[s]={ys}, y[t]={yt}")


node_id=1708, incident edges count = 12

=== node 1708 Top-15 incident edges by edge_mask ===
01. eid= 3471  (1708 ->  873)  w=0.9098   y[s]=3, y[t]=0
02. eid= 7558  (1708 -> 1857)  w=0.8880   y[s]=3, y[t]=3
03. eid= 1925  (1708 ->  467)  w=0.8830   y[s]=3, y[t]=0
04. eid= 9511  (1708 -> 2314)  w=0.8776   y[s]=3, y[t]=3
05. eid= 9509  (1708 -> 2313)  w=0.8742   y[s]=3, y[t]=2
06. eid= 6845  ( 873 -> 1708)  w=0.8566   y[s]=0, y[t]=3
07. eid= 6849  (2314 -> 1708)  w=0.8521   y[s]=3, y[t]=3
08. eid= 6848  (2313 -> 1708)  w=0.8500   y[s]=2, y[t]=3
09. eid= 6847  (1857 -> 1708)  w=0.8393   y[s]=3, y[t]=3
10. eid= 6844  ( 467 -> 1708)  w=0.8236   y[s]=0, y[t]=3
11. eid= 6846  (1358 -> 1708)  w=0.2138   y[s]=2, y[t]=3
12. eid= 5366  (1708 -> 1358)  w=0.1066   y[s]=3, y[t]=2


incident edges count = ?
表示 1708 这个节点在图里连接了多少条边（度数相关，注意 Cora 的边通常是有向存储/双向存储，所以数值会偏大）

Top-k 列表里每一行：

(s -> t)：边两端节点编号

w=...：解释器认为这条边对 1708 的预测贡献的重要程度（越大越关键）

y[s], y[t]：两端节点的真实类别（0-6）

Cora 在 PyG 里通常把无向边存成两条有向边（u→v 和 v→u）。
所以 incident edges count=12 对应大概 6 个真实邻居节点：873, 1857, 467, 2314, 2313, 1358

1708 的“关键邻居”里，异类占多数（4/6）：

同类（=3）：1857, 2314 （2 个）
异类（≠3）：873(0), 467(0), 2313(2), 1358(2)（4 个）

1708 的决策主要依赖与 (0类、2类) 邻居的高权重边，这些边可能是误判的重要结构原因

7、把“邻居标签混杂”量化成统计结果，统计关键邻居的标签构成 + 权重占比（去掉双向重复）

In [13]:
import torch
from collections import defaultdict

node_id = 1708
edge_index = data.edge_index
edge_mask = explanation.edge_mask  # 需要和你当前变量一致

src, dst = edge_index[0], edge_index[1]
incident = (src == node_id) | (dst == node_id)
incident_eids = incident.nonzero(as_tuple=False).view(-1)

# 把 incident edges 映射到 “邻居节点”
neighbors = []
weights = []
for eid in incident_eids.tolist():
    s = int(src[eid]); t = int(dst[eid])
    nb = t if s == node_id else s
    neighbors.append(nb)
    weights.append(float(edge_mask[eid]))

# 去重（因为无向边被存成两条有向边）
# 规则：同一个邻居 nb 可能出现两次（u->v,v->u），我们取其中更大的权重代表这条“无向连接”的重要性
best_w = {}
for nb, w in zip(neighbors, weights):
    if nb not in best_w or w > best_w[nb]:
        best_w[nb] = w

true_y = int(data.y[node_id])
label_count = defaultdict(int)
label_weight = defaultdict(float)

for nb, w in best_w.items():
    lab = int(data.y[nb])
    label_count[lab] += 1
    label_weight[lab] += w

total_neighbors = len(best_w)
total_weight = sum(best_w.values())

print(f"node_id={node_id}, true_y={true_y}")
print(f"unique neighbors (undirected) = {total_neighbors}")
print("\nLabel counts among key neighbors:")
for lab in sorted(label_count):
    print(f"  label {lab}: {label_count[lab]}")

print("\nWeight share among key neighbors:")
for lab in sorted(label_weight):
    print(f"  label {lab}: weight_sum={label_weight[lab]:.4f}, share={label_weight[lab]/total_weight:.2%}")

same_class = label_count.get(true_y, 0)
print(f"\nHomophily (by count) = {same_class}/{total_neighbors} = {same_class/total_neighbors:.2%}")


node_id=1708, true_y=3
unique neighbors (undirected) = 6

Label counts among key neighbors:
  label 0: 2
  label 2: 2
  label 3: 2

Weight share among key neighbors:
  label 0: weight_sum=1.7928, share=38.59%
  label 2: weight_sum=1.0880, share=23.42%
  label 3: weight_sum=1.7655, share=38.00%

Homophily (by count) = 2/6 = 33.33%


Label counts 是“6 个关键邻居分别属于哪些真实类别”

Weight share 是“这 6 个关键邻居对解释权重的贡献占比”

33.33%
在解释器认为最关键的邻居中，只有 1/3 和 1708 同类（真实类=3）。
GCN 的信息是“聚合邻居”的，所以当关键邻居同质性低时，节点表示很容易被“拉向”其他类别特征，形成误判风险。

权重贡献几乎是“三分天下”

Weight share：

label 0：38.59%
label 2：23.42%
label 3：38.00%

这意味着：在解释器看来，支持 0 类邻居的边（38.59%）和支持 3 类邻居的边（38.00%）贡献几乎一样大。
这个节点的“关键邻域证据”是冲突的——模型既接收到强烈的“你像 3 类”的邻域信号，也接收到同样强烈的“你像 0 类”的邻域信号

8、deletion Fidelity（删掉关键边，看预测掉不掉）
目标：验证解释器说“重要”的边，是否真的对模型决策有因果影响。
做法：只在图里移除 node_id 的 top-k 关键无向邻居边（双向两条一起删）

In [14]:
import torch
import torch.nn.functional as F

node_id = 1708
model.eval()

@torch.no_grad()
def predict_conf(edge_index_new):
    out = model(data.x, edge_index_new)
    prob = F.softmax(out, dim=1)
    pred = int(prob[node_id].argmax())
    conf = float(prob[node_id, pred])
    return out, prob, pred, conf

# 1) baseline
_, prob0, pred0, conf0 = predict_conf(data.edge_index)
true_y = int(data.y[node_id])
print("BASE:", {"node": node_id, "true": true_y, "pred_before": pred0, "conf_before": conf0})

# 2) 取解释器给的邻居重要性（用你上一段 best_w 的逻辑）
src, dst = data.edge_index[0], data.edge_index[1]
incident = (src == node_id) | (dst == node_id)
incident_eids = incident.nonzero(as_tuple=False).view(-1)

best_w = {}  # nb -> w
for eid in incident_eids.tolist():
    s = int(src[eid]); t = int(dst[eid])
    nb = t if s == node_id else s
    w = float(explanation.edge_mask[eid])
    if nb not in best_w or w > best_w[nb]:
        best_w[nb] = w

# 3) 选择 top-k 邻居（无向）
def remove_topk_neighbors(k):
    top_nbs = sorted(best_w.items(), key=lambda x: x[1], reverse=True)[:k]
    top_nbs = {nb for nb, _ in top_nbs}

    # 删除所有 (node_id <-> nb) 的双向边（以及可能存在的单向）
    keep = []
    for i in range(data.edge_index.size(1)):
        s = int(src[i]); t = int(dst[i])
        if (s == node_id and t in top_nbs) or (t == node_id and s in top_nbs):
            continue
        keep.append(i)
    keep = torch.tensor(keep, dtype=torch.long)

    edge_index_new = data.edge_index[:, keep]
    return edge_index_new, top_nbs, int(keep.numel())

for k in [1, 2, 3, 4, 6]:
    edge_new, top_nbs, m = remove_topk_neighbors(k)
    _, prob1, pred1, conf1 = predict_conf(edge_new)
    conf_on_old = float(prob1[node_id, pred0])
    print(f"\nDeletion top-{k} neighbors:")
    print({
        "removed_neighbors": sorted(list(top_nbs)),
        "edges_after": m,
        "pred_after": pred1,
        "conf_after": conf1,
        "conf_after_on_pred_before": conf_on_old,
        "drop": conf0 - conf_on_old
    })


BASE: {'node': 1708, 'true': 3, 'pred_before': 1, 'conf_before': 0.20795901119709015}

Deletion top-1 neighbors:
{'removed_neighbors': [873], 'edges_after': 10554, 'pred_after': 1, 'conf_after': 0.20052409172058105, 'conf_after_on_pred_before': 0.20052409172058105, 'drop': 0.007434919476509094}

Deletion top-2 neighbors:
{'removed_neighbors': [873, 1857], 'edges_after': 10552, 'pred_after': 1, 'conf_after': 0.16885067522525787, 'conf_after_on_pred_before': 0.16885067522525787, 'drop': 0.039108335971832275}

Deletion top-3 neighbors:
{'removed_neighbors': [467, 873, 1857], 'edges_after': 10550, 'pred_after': 2, 'conf_after': 0.23668847978115082, 'conf_after_on_pred_before': 0.17294177412986755, 'drop': 0.035017237067222595}

Deletion top-4 neighbors:
{'removed_neighbors': [467, 873, 1857, 2314], 'edges_after': 10548, 'pred_after': 2, 'conf_after': 0.37168121337890625, 'conf_after_on_pred_before': 0.132615327835083, 'drop': 0.07534368336200714}

Deletion top-6 neighbors:
{'removed_neighb

对节点 1708，把解释器给出的 top-k 关键邻居对应的边（双向）删掉，然后重新 forward

以 Deletion top-6 neighbors 为例：

removed_neighbors: 你删掉的 top-k 关键邻居（这里是 6 个：467, 873, 1358, 1857, 2313, 2314）

edges_after: 删掉这些邻居相关的 incident edges 后，全图还剩多少边（从 10556 变成 10544，说明一共删了 12 条边，符合“无向边两条有向存储”）

pred_after / conf_after: 在删边后的图上，节点 1708 的新预测类别和该类别概率

conf_after_on_pred_before: 删边后，节点对“原预测类别 pred_before=1”的概率

drop = conf_before - conf_after_on_pred_before: 关键指标，表示删掉解释边后，对原预测的支撑力度下降了多少

关键结论：删边会逐步“纠正”错误预测（说明解释边确实影响决策）
BASE 是：
True = 3，但 Pred_before = 1，conf_before = 0.208（本来就不自信，属于“勉强选了 1 类”）

删除少量关键邻居（top-1 / top-2）：影响很小

top-1（删 873）：drop 只有 0.0074，预测仍是 1
说明 单独删最关键的一个邻居不足以改变决策。

top-2（删 873,1857）：drop 0.0391，预测仍是 1
支撑 1 类的证据在减弱，但还没到翻转。

删除到 top-3 / top-4：预测开始翻到 2 类

top-3（再删 467）：pred_after 变成 2

top-4（再删 2314）：pred_after 仍为 2，且 conf_after 升到 0.372
删掉更多“关键邻居”后，模型不再坚持 1 类，而是被另一组结构证据带向 2 类

删除到 top-6：预测终于回到真实类 3

top-6：pred_after = 3（纠正了），conf_after = 0.300
同时对原错误类别 1 的概率降到 0.133（drop ≈ 0.075）
edge_mask 选出来的这批邻居边，确实在支撑模型的错误决策（pred=1）
错误并不是来自某一条边，而更像是多个关键邻居共同“拉偏”
当你把这组“拉偏证据”剔除后，剩余邻域信息反而更支持真实类 3

10、做 Insertion Fidelity（只保留 top-k 邻居边，看还能不能复现决策）
Insertion / Sufficiency：只保留解释认为关键的 top-k 邻居边（其余边都去掉），看看模型能否仍做出相同预测/保持高置信度。

In [15]:
import torch
from torch_geometric.data import Data

node_id = 1708
top_k = 6  # 你可以之后试 2/4/8

# 原始信息
edge_index = data.edge_index
edge_mask = explanation.edge_mask
x = data.x

# 找到与 node_id 相连的边
src, dst = edge_index[0], edge_index[1]
incident = (src == node_id) | (dst == node_id)
incident_eids = incident.nonzero(as_tuple=False).view(-1)

# 按 edge_mask 排序，取 Top-k
scores = edge_mask[incident_eids]
order = torch.argsort(scores, descending=True)
keep_eids = incident_eids[order[:top_k]]

# 构造“只保留这些边”的 edge_index
new_edge_index = edge_index[:, keep_eids]

print(f"Keep {top_k} edges for node {node_id}")
print(f"Original edges: {edge_index.size(1)}  ->  After insertion: {new_edge_index.size(1)}")

# 用这个子图重新做一次 forward
model.eval()
out_before = model(x, edge_index)
out_after = model(x, new_edge_index)

# 概率
prob_before = out_before.softmax(dim=1)
prob_after = out_after.softmax(dim=1)

pred_before = prob_before[node_id].argmax().item()
conf_before = prob_before[node_id, pred_before].item()

pred_after = prob_after[node_id].argmax().item()
conf_after = prob_after[node_id, pred_after].item()
conf_after_on_pred_before = prob_after[node_id, pred_before].item()

print("\n=== Insertion result ===")
print({
    "node": node_id,
    "true": int(data.y[node_id]),
    "pred_before": pred_before,
    "conf_before": conf_before,
    "pred_after": pred_after,
    "conf_after": conf_after,
    "conf_after_on_pred_before": conf_after_on_pred_before,
    "kept_edges": top_k
})


Keep 6 edges for node 1708
Original edges: 10556  ->  After insertion: 6

=== Insertion result ===
{'node': 1708, 'true': 3, 'pred_before': 1, 'conf_before': 0.20795901119709015, 'pred_after': 0, 'conf_after': 0.4241393208503723, 'conf_after_on_pred_before': 0.1891966313123703, 'kept_edges': 6}


原图边数：10556
只保留 node 1708 的 top-6 解释边后：只剩 6 条边
原图预测：pred_before = 1，且置信度 conf_before = 0.2079（很低）

只保留 top-6 解释边后：预测变成 pred_after = 0，置信度 conf_after = 0.4241（反而更高）

同时，“对原预测类别 1 的置信度”变成 conf_after_on_pred_before = 0.1892（比 0.2079 还低一点）

conf_before = 0.2079 非常低，说明模型在全图里对它的预测（类 1）本身就不稳定，属于“摇摆/边界点”

“只剩 6 条边”会把 GCN 的信息传播机制彻底改变

GCN 的两层卷积本质上是在做邻域聚合。
你把全图删到只剩 6 条边，相当于把 node 1708 的“信息来源”强行限制为极少数路径：

原图时：1708 的表示会受到更广的图结构影响（尤其是 2-hop 传播）

只留 6 条边：模型只能看到极少的邻居信息 → 表示空间会被拉到另一类（这里变成了 class 0）
对于误分类节点 1708，解释器给出的 top-k 解释边并不能构成一个“sufficient explanation”（不足以单独支撑模型原预测）；模型的原决策更像是由更大范围的图上下文共同决定，且该节点本身预测置信度低、决策边界不稳定

11、对照组1709（预测正确的节点）做同样的 insertion

12、对 node 1709 生成 edge_mask（边重要性）

In [16]:
node_id = 1709

model.eval()
ex_1709 = explainer(
    x=data.x,
    edge_index=data.edge_index,
    index=node_id,
)

print("node_id =", node_id)
print("has edge_mask:", ex_1709.edge_mask is not None)
print("edge_mask shape:", tuple(ex_1709.edge_mask.shape))
print("edge_index shape:", tuple(data.edge_index.shape))


node_id = 1709
has edge_mask: True
edge_mask shape: (10556,)
edge_index shape: (2, 10556)


13、列出 node 1709 的“关键边”（incident edges Top-K）

In [17]:
import torch

node_id = 1709
edge_index = data.edge_index
edge_mask = ex_1709.edge_mask.detach().cpu()

# 1) 找到所有“与 node_id 相连”的边（有向：src==node 或 dst==node）
src, dst = edge_index[0], edge_index[1]
incident_eids = torch.where((src == node_id) | (dst == node_id))[0]

print(f"node_id={node_id}, incident edges count = {incident_eids.numel()}")

# 2) 对这些边按 edge_mask 排序（从重要到不重要）
scores = edge_mask[incident_eids]
order = torch.argsort(scores, descending=True)
top_eids = incident_eids[order]

# 3) 打印 Top-15 关键边：eid, (u->v), 权重w, 以及两端真实标签
topk = min(15, top_eids.numel())
print(f"\n=== node {node_id} Top-{topk} incident edges by edge_mask ===")
for i in range(topk):
    eid = int(top_eids[i])
    u = int(src[eid])
    v = int(dst[eid])
    w = float(edge_mask[eid])
    print(f"{i+1:02d}. eid={eid:5d}  ({u:4d} -> {v:4d})  w={w:.4f}   y[s]={int(data.y[u])}, y[t]={int(data.y[v])}")


node_id=1709, incident edges count = 10

=== node 1709 Top-10 incident edges by edge_mask ===
01. eid= 6852  (1739 -> 1709)  w=0.8509   y[s]=2, y[t]=2
02. eid= 6851  (1738 -> 1709)  w=0.8390   y[s]=2, y[t]=2
03. eid= 6854  (2365 -> 1709)  w=0.8309   y[s]=2, y[t]=2
04. eid= 6850  (1358 -> 1709)  w=0.7990   y[s]=2, y[t]=2
05. eid= 6853  (1986 -> 1709)  w=0.1868   y[s]=3, y[t]=2
06. eid= 6991  (1709 -> 1738)  w=0.1276   y[s]=2, y[t]=2
07. eid= 9673  (1709 -> 2365)  w=0.1251   y[s]=2, y[t]=2
08. eid= 6998  (1709 -> 1739)  w=0.1179   y[s]=2, y[t]=2
09. eid= 8167  (1709 -> 1986)  w=0.1171   y[s]=2, y[t]=3
10. eid= 5367  (1709 -> 1358)  w=0.1155   y[s]=2, y[t]=2


1709 在图里一共和 10 条有向边相连
Top-4 的边权重都在 0.80+，并且这些边的邻居标签全部是 2，和目标节点 1709 的标签 2 一致。

只有一组邻居 1986 (label=3) 相关边（eid=6853 / 8167），但权重明显低（0.1868/0.1171），说明它对决策影响较小，甚至可能是噪声/干扰边。

14、统计关键邻居的标签分布、权重占比、同质性（homophily）
1709 的解释边主要连向哪些类别的邻居？同类邻居占比高不高？

In [18]:
import torch
from collections import Counter, defaultdict

node_id = 1709
edge_index = data.edge_index
edge_mask = ex_1709.edge_mask.detach().cpu()

src, dst = edge_index[0], edge_index[1]
incident_eids = torch.where((src == node_id) | (dst == node_id))[0]

# 1) “邻居”去重：无向意义下，1709连到哪些不同的邻居节点
neighbors = []
for eid in incident_eids.tolist():
    u = int(src[eid]); v = int(dst[eid])
    nb = v if u == node_id else u
    neighbors.append(nb)
uniq_neighbors = sorted(set(neighbors))

true_y = int(data.y[node_id])
print(f"node_id={node_id}, true_y={true_y}")
print(f"unique neighbors (undirected) = {len(uniq_neighbors)}")

# 2) 邻居标签计数
label_counts = Counter(int(data.y[n]) for n in uniq_neighbors)
print("\nLabel counts among neighbors:")
for lab, cnt in sorted(label_counts.items()):
    print(f"  label {lab}: {cnt}")

# 3) 权重占比（把 incident edges 的 edge_mask 按“邻居标签”累加）
weight_sum_by_label = defaultdict(float)
total_w = 0.0

for eid in incident_eids.tolist():
    u = int(src[eid]); v = int(dst[eid])
    nb = v if u == node_id else u
    lab = int(data.y[nb])
    w = float(edge_mask[eid])
    weight_sum_by_label[lab] += w
    total_w += w

print("\nWeight share among incident edges (by neighbor label):")
for lab, wsum in sorted(weight_sum_by_label.items()):
    share = (wsum / total_w) if total_w > 0 else 0.0
    print(f"  label {lab}: weight_sum={wsum:.4f}, share={share:.2%}")

# 4) 同质性（按邻居“数量”口径）
same = sum(1 for n in uniq_neighbors if int(data.y[n]) == true_y)
homophily = same / len(uniq_neighbors) if len(uniq_neighbors) > 0 else 0.0
print(f"\nHomophily (by count) = {same}/{len(uniq_neighbors)} = {homophily:.2%}")


node_id=1709, true_y=2
unique neighbors (undirected) = 5

Label counts among neighbors:
  label 2: 4
  label 3: 1

Weight share among incident edges (by neighbor label):
  label 2: weight_sum=3.8059, share=92.60%
  label 3: weight_sum=0.3040, share=7.40%

Homophily (by count) = 4/5 = 80.00%


1709 的预测主要依赖同类邻居（label=2）
1709 的无向邻居一共 5 个，其中 4 个是 label=2、1 个是 label=3
所以按“邻居数量口径”的同质性（homophily）是 80%

解释边的权重几乎都“投给”了同类邻居

label=2 的邻居贡献了 92.60% 的解释权重
label=3 的邻居只占 7.40% 的解释权重
不是仅仅“邻居里同类多”，而是模型真正用来做决策的那些边（edge_mask 高的边）几乎全部指向 label=2

15、Deletion fidelity（删除关键邻居，看预测是否崩）
如果我们删掉解释认为最重要的邻居（或边），模型对原预测类别的置信度应该明显下降，甚至改判

In [19]:
import torch
import copy

def get_pred_and_conf(model, x, edge_index, node_id):
    model.eval()
    out = model(x, edge_index)
    prob = out.softmax(dim=1)
    pred = int(prob[node_id].argmax())
    conf = float(prob[node_id, pred])
    return pred, conf, prob[node_id].detach().cpu()

def delete_neighbors(edge_index, node_id, remove_neighbors_set):
    src, dst = edge_index
    keep = torch.ones(src.size(0), dtype=torch.bool)

    for i in range(src.size(0)):
        u = int(src[i]); v = int(dst[i])
        if u == node_id and v in remove_neighbors_set:
            keep[i] = False
        elif v == node_id and u in remove_neighbors_set:
            keep[i] = False

    return edge_index[:, keep], int(keep.sum())

node_id = 1709
edge_mask = ex_1709.edge_mask.detach().cpu()
edge_index = data.edge_index

# 1) 基线预测
pred_before, conf_before, prob_before = get_pred_and_conf(model, data.x, edge_index, node_id)
true_y = int(data.y[node_id])

BASE = {"node": node_id, "true": true_y, "pred_before": pred_before, "conf_before": conf_before}
print("BASE:", BASE)

# 2) 找 incident edges，并按 edge_mask 找“最重要的邻居”
src, dst = edge_index[0], edge_index[1]
incident = torch.where((src == node_id) | (dst == node_id))[0]

# 把 edge 权重聚合到“邻居”层面：一个邻居可能有两条方向边
nb_weight = {}
for eid in incident.tolist():
    u = int(src[eid]); v = int(dst[eid])
    nb = v if u == node_id else u
    nb_weight[nb] = nb_weight.get(nb, 0.0) + float(edge_mask[eid])

# 邻居按重要性排序
sorted_nbs = sorted(nb_weight.items(), key=lambda x: x[1], reverse=True)
sorted_nbs = [nb for nb, w in sorted_nbs]

print("\nNeighbors ranked by importance (neighbor-level sum of edge_mask):")
for i, nb in enumerate(sorted_nbs, 1):
    print(f"{i:02d}. nb={nb:4d}  y={int(data.y[nb])}  weight_sum={nb_weight[nb]:.4f}")

# 3) 逐步删除 top-m 邻居
for m in [1, 2, 3, 4, 5]:
    remove = set(sorted_nbs[:m])
    new_ei, edges_after = delete_neighbors(edge_index, node_id, remove)

    pred_after, conf_after, prob_after = get_pred_and_conf(model, data.x, new_ei, node_id)

    # 关注：原预测类别 pred_before 的置信度掉了多少
    conf_after_on_pred_before = float(prob_after[pred_before])
    drop = conf_before - conf_after_on_pred_before

    print(f"\nDeletion top-{m} neighbors:")
    print({
        "removed_neighbors": list(remove),
        "edges_after": edges_after,
        "pred_after": pred_after,
        "conf_after": conf_after,
        "conf_after_on_pred_before": conf_after_on_pred_before,
        "drop": drop,
    })


C:\Users\Administrator\AppData\Local\Temp\ipykernel_2804\439430815.py:9: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:836.)
  conf = float(prob[node_id, pred])


BASE: {'node': 1709, 'true': 2, 'pred_before': 2, 'conf_before': 0.8195142149925232}

Neighbors ranked by importance (neighbor-level sum of edge_mask):
01. nb=1739  y=2  weight_sum=0.9689
02. nb=1738  y=2  weight_sum=0.9665
03. nb=2365  y=2  weight_sum=0.9560
04. nb=1358  y=2  weight_sum=0.9145
05. nb=1986  y=3  weight_sum=0.3040

Deletion top-1 neighbors:
{'removed_neighbors': [1739], 'edges_after': 10554, 'pred_after': 2, 'conf_after': 0.7844098806381226, 'conf_after_on_pred_before': 0.7844098806381226, 'drop': 0.035104334354400635}

Deletion top-2 neighbors:
{'removed_neighbors': [1738, 1739], 'edges_after': 10552, 'pred_after': 2, 'conf_after': 0.7038491368293762, 'conf_after_on_pred_before': 0.7038491368293762, 'drop': 0.11566507816314697}

Deletion top-3 neighbors:
{'removed_neighbors': [1738, 1739, 2365], 'edges_after': 10550, 'pred_after': 2, 'conf_after': 0.3582023084163666, 'conf_after_on_pred_before': 0.3582023084163666, 'drop': 0.4613119065761566}

Deletion top-4 neighbors:

BASE是：
node=1709，真实标签 true=2
模型预测 pred_before=2（预测正确）
对类别 2 的置信度 conf_before=0.8195

邻居重要性排名前四个关键邻居全是同类（label=2）
唯一的异类邻居（label=3）重要性明显更低（0.3040）

删除 top-m 关键邻居后
删 1 个（1739）：置信度 0.8195 → 0.7844（小降，drop=0.0351），预测仍是 2

删 2 个（1738,1739）：置信度 0.7038（drop=0.1157），仍是 2

删 3 个（再删 2365）：置信度直接掉到 0.3582（drop=0.4613），仍是 2，但已经很虚

删 4 个（再删 1358）：预测直接翻转到 3，并且你关注的“原预测类2的置信度”只剩 0.1649（drop=0.6546）

删 5 个（把异类 1986 也删了）：预测仍是 3，对原类2的置信度进一步掉到 0.1053

删到 3 个同类关键邻居时，原类置信度已经“腰斩再腰斩”
删到 4 个同类关键邻居时，模型直接从 2 改判为 3
1709 的正确预测高度依赖于这四个同类关键邻居提供的结构证据

16、Insertion Fidelity（只保留 top-k 邻居/边，能否支撑预测）

In [20]:
import torch

def keep_only_neighbors(edge_index, node_id, keep_neighbors_set):
    src, dst = edge_index
    keep = torch.zeros(src.size(0), dtype=torch.bool)

    for i in range(src.size(0)):
        u = int(src[i]); v = int(dst[i])
        if u == node_id and v in keep_neighbors_set:
            keep[i] = True
        elif v == node_id and u in keep_neighbors_set:
            keep[i] = True

    return edge_index[:, keep], int(keep.sum())

node_id = 1709

# 基线
pred_before, conf_before, prob_before = get_pred_and_conf(model, data.x, data.edge_index, node_id)
true_y = int(data.y[node_id])
print("BASE:", {"node": node_id, "true": true_y, "pred_before": pred_before, "conf_before": conf_before})

# 你的邻居排序已经有了：sorted_nbs
# 这里按 top-k 邻居做 insertion
for k in [1, 2, 3, 4, 5]:
    keep_nbs = set(sorted_nbs[:k])
    new_ei, edges_after = keep_only_neighbors(data.edge_index, node_id, keep_nbs)

    pred_after, conf_after, prob_after = get_pred_and_conf(model, data.x, new_ei, node_id)
    conf_after_on_pred_before = float(prob_after[pred_before])  # 原预测类的置信度

    print(f"\nInsertion keep top-{k} neighbors:")
    print({
        "kept_neighbors": list(keep_nbs),
        "edges_after": edges_after,
        "pred_after": pred_after,
        "conf_after": conf_after,
        "conf_after_on_pred_before": conf_after_on_pred_before,
    })


BASE: {'node': 1709, 'true': 2, 'pred_before': 2, 'conf_before': 0.8195142149925232}

Insertion keep top-1 neighbors:
{'kept_neighbors': [1739], 'edges_after': 2, 'pred_after': 2, 'conf_after': 0.6025946736335754, 'conf_after_on_pred_before': 0.6025946736335754}

Insertion keep top-2 neighbors:
{'kept_neighbors': [1738, 1739], 'edges_after': 4, 'pred_after': 2, 'conf_after': 0.7197235822677612, 'conf_after_on_pred_before': 0.7197235822677612}

Insertion keep top-3 neighbors:
{'kept_neighbors': [1738, 1739, 2365], 'edges_after': 6, 'pred_after': 2, 'conf_after': 0.9425157904624939, 'conf_after_on_pred_before': 0.9425157904624939}

Insertion keep top-4 neighbors:
{'kept_neighbors': [1738, 1739, 2365, 1358], 'edges_after': 8, 'pred_after': 2, 'conf_after': 0.9601922035217285, 'conf_after_on_pred_before': 0.9601922035217285}

Insertion keep top-5 neighbors:
{'kept_neighbors': [1986, 1738, 1739, 1358, 2365], 'edges_after': 10, 'pred_after': 2, 'conf_after': 0.9286341071128845, 'conf_after_o

BASE是：
1709 预测为 2，模型对类别 2 的概率大约 0.82

只留 1 个关键邻居
只靠 一个邻居 1739（对应 2 条有向边：1709↔1739），模型仍预测 2。
但置信度从 0.82 降到 0.60：说明证据足以维持判别方向，但信息量不足以非常确定

只留 2 个关键邻居
加入第二个关键邻居（1738）后，置信度明显上升到 0.72。
预测仍稳定为 2，模型对 2 类的信心，随着关键同类邻居数量增加而上升，符合 GCN 的“邻域聚合强化同类信号”的机制

只留 3 个关键邻居
只保留 top-3 关键邻居（1738、1739、2365），模型对类别 2 的置信度达到 0.94，比 BASE 的 0.82 还高！！
这些 top-3 邻居几乎就构成了模型判别 1709 的“核心证据子图”。
完整图里还有大量其它边/邻居带来的信息，对 1709 来说要么是噪声、要么是稀释（dilute）信号。
只保留核心证据，反而让模型更“专注”，概率更高

留 4 个关键邻居
再加一个同类关键邻居 1358，置信度到 0.96。
这基本说明：1709 的预测几乎完全由这些同类关键邻居支撑

留 5 个关键邻居（加入一个异类邻居 1986）
加入第 5 个邻居（1986）后，置信度从 0.9602 下降到 0.9286（仍高，但明显变低）
1986 这个异类邻居不是“支持类别 2”的证据，反而会引入一部分竞争信号，使模型对 2 的确信程度下降。
这与 deletion 那边看到的现象一致：当删除掉关键同类邻居后，模型更容易被其它类别拉走

